# Processamento de Linguagem Natural - Trabalho Prático 1
### Thaís Ferreira da Silva - 2021092571

### Objetivo
O objetivo desse trabalho é entender e aplicar conceitos de word embeddings, analisando a qualidade dos modelos treinados em função dos parâmetros escolhidos para o seu treinamento.

### Atividades Principais
Para esse trabalho, utilizei o gensim no treinamento do modelo atraves da variação dos parâmetros do Word2Vec.

Dividi esse trabalho em 4 partes principais:

##### Funções auxiliares: 
Parte contendo todas os métodos implementados para as partes seguintes

##### Treinamento: 
Aqui defini os parametros de treinamento e realizei de fato o treinamento dos 54 modelos resultado da variação dos parâmetos: 
- tamanho do vetor de palavras
- tamanho da janela de contexto
- número de interações
- utiliza CBOW ou skip-gram. 

Realizei o treinamento 2 vezes com parâmetros diferentes e armazenei em 2 pastas separadas para futuras analises de desempenho.

##### Avaliação: 
Nessa parte utilizei a distância média do cosseno (metrica exigida pelo trabalho) e a acurácia do modelo para escolher o melhor modelo treinado de acordo com as predições sobre analogias.


### Import

In [102]:
# Imports do gensim - para word2vec
import gensim
from gensim.models import Word2Vec
from gensim.models.word2vec import Text8Corpus

# Imports para graficos
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Imports extras
from scipy.spatial.distance import cosine
import os
import random
import time

-------------------------------------------
## Funções Auxiliares

Optei por desenvolver 4 principais funções com os seguintes objetivos:

- Gerar as combinaçoes de hiperparametros: São 54 variações, onde 3 parâmetros possuem 3 variações, e o for fim temos a variação do skip-gram e CBOW

- Treinar e salvar os modelos: Os modelos são treinados com Word2Vac e para facilitar as analises após o treinamento de cada modelo, eles são salvos em 2 pastas diferentes

- Pegar as analogias: Aqui o arquivo de texto com as analogias são lidos de 4 em 4 palavras e armazenados em um vetor que posteriormente será utilizado para a previsão e avaliação dos modelos

- Avaliar o model: Na avaliação dos modelos, é necessário passar por todas as analogias, e realizar a operação vetorial e prever o 4° elemento do vetor. Dessa forma, ao final de todo o processamento temos os calculos das métricas de avaliação, sendo elas a distância de cosseno e a acurácia.

In [68]:
def generate_hyperparameter_combinations(param_grid):
    from itertools import product
    keys, values = zip(*param_grid.items())
    combinations = [dict(zip(keys, v)) for v in product(*values)]
    return combinations


In [69]:
def train_and_save_model(corpus, params, output_dir):
    model_name = f"word2vec_vs{params['vector_size']}_win{params['window']}_sg{params['sg']}_ep{params['epochs']}"
    print(f"Treinando modelo: {model_name}")
    
    model = Word2Vec(
        sentences=corpus,
        vector_size=params['vector_size'],
        window=params['window'],
        sg=params['sg'],
        epochs=params['epochs'],
        workers=8 #cores do processador
    )
    
    model_path = os.path.join(output_dir, f"{model_name}.model")
    model.save(model_path)
    print(f"Modelo salvo: {model_path}")
    
    return model

In [70]:
def get_analogies(model, questions_words_path):
    final_analogies = []
    with open(questions_words_path, 'r') as f:
        for line in f:
            if line.startswith(':'):
                continue

            words = [word.lower().strip() for word in line.split()]

            final_words = [word for word in words if word in model.wv]
            if len(final_words) != 4:
                continue
            
            final_analogies.append(final_words)
    
    random.shuffle(final_analogies)
    return final_analogies

In [71]:
def evaluate_models(model, analogies):
    avg_distance = 0
    correct = 0
    count = 0
    accuracy = 0

    for analogy in analogies:
        if len(analogy) == 4:
            count += 1

            try:
                result_vector = model.wv[analogy[1]] - model.wv[analogy[0]] + model.wv[analogy[2]]

                predicted = model.wv.similar_by_vector(result_vector, topn=20, restrict_vocab=None)
                predicted_word = next((word for word, _ in predicted if word not in analogy[:3]), None)

                if predicted_word == analogy[3]:
                    correct += 1

                if predicted_word in model.wv:
                    cosine_distance = cosine(model.wv[analogy[3]], model.wv[predicted_word])
                    avg_distance += cosine_distance

            except KeyError as e:
                print(e)

    accuracy = correct / count if count > 0 else 0
    avg_distance = avg_distance / count if count > 0 else 0

    return accuracy, avg_distance

In [146]:
def new_evaluate_models(model, analogies, model_name):
    avg_distance = 0  # Média da distância cosseno
    total_loss = 0  # Loss total (pode ser distância cosseno ou MSE)
    correct = 0  # Contagem de analogias corretas
    count = 0  # Total de analogias processadas
    results = []  # Lista para armazenar os detalhes de cada analogia
    start_time = time.time()

    for analogy in analogies:
        if len(analogy) == 4:
            count += 1

            try:
                # Calcula o vetor resultante da analogia
                result_vector = model.wv[analogy[1]] - model.wv[analogy[0]] + model.wv[analogy[2]]

                # Prediz a palavra com base no vetor calculado
                predicted = model.wv.similar_by_vector(result_vector, topn=20, restrict_vocab=None)
                predicted_word = next((word for word, _ in predicted if word not in analogy[:3]), None)

                # Verifica se a previsão está correta
                is_correct = predicted_word == analogy[3]
                if is_correct:
                    correct += 1

                # Calcula a distância cosseno entre a palavra esperada e a palavra predita
                if predicted_word in model.wv:
                    cosine_distance = cosine(model.wv[analogy[3]], model.wv[predicted_word])
                    avg_distance += cosine_distance

                # Calcula a loss como a distância cosseno entre o vetor resultante e o vetor esperado
                if analogy[3] in model.wv:
                    cosine_loss = cosine(model.wv[analogy[3]], result_vector)  # Loss usando distância cosseno
                    total_loss += cosine_loss

                # Armazena os detalhes da analogia, incluindo todas as palavras preditas
                results.append({
                    'sg': 0,
                    'vector_size': 0,
                    'window': 0,
                    'epochs': 0,
                    'analogy': f'{analogy[0]}:{analogy[1]}::{analogy[2]}:{analogy[3]}',
                    'word_a': analogy[0],
                    'word_b': analogy[1],
                    'word_c': analogy[2],
                    'expected_word': analogy[3],
                    'predicted_word': predicted_word,
                    'all_predicted_words': [word for word, _ in predicted],  # Lista de todas as palavras preditas
                    'correct': is_correct,
                    'accuracy': 0,
                    'avg_distance': 0,
                    'cosine_loss': cosine_loss  # Loss para esta analogia
                })

            except KeyError as e:
                print(f"Palavra não encontrada no vocabulário: {e}")

    end_time = time.time()
    execution_time = end_time - start_time

    # Calcula as métricas globais
    accuracy = correct / count if count > 0 else 0
    avg_distance = avg_distance / count if count > 0 else 0
    avg_loss = total_loss / count if count > 0 else 0  # Calcula a média da loss para todas as analogias

    # Converte a lista de resultados em um DataFrame
    df_results = pd.DataFrame(results)

    return accuracy, avg_distance, avg_loss, df_results

In [168]:
def plot_CBOWxSkipGram(df_final):
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    sns.countplot(data=df_final, x='sg', hue='correct', ax=axes[0])
    axes[0].set_title('Número de acertos e erros por tipo de sg')

    sns.scatterplot(data=df_final, x='accuracy', y='avg_distance', hue='sg', ax=axes[1])
    axes[1].set_title('Acurácia x Distância Média')

    plt.tight_layout()
    plt.show()

def plot_embeddings_impact(df_final):
    #plotar e mostrar o impacto do tamanho do embedding no valor da acurácia e da distância média
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    sns.scatterplot(data=df_final, x='vector_size', y='accuracy', hue='sg', ax=axes[0])
    axes[0].set_title('Tamanho do Embedding x Acurácia')

    sns.scatterplot(data=df_final, x='vector_size', y='avg_distance', hue='sg', ax=axes[1])
    axes[1].set_title('Tamanho do Embedding x Distância Média')

    plt.tight_layout()
    plt.show()

    #Gráfico de Linhas (Evolução da Acurácia e Distância Média)
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    sns.lineplot(data=df_final, x='epochs', y='accuracy', hue='sg', ax=axes[0])
    axes[0].set_title('Evolução da Acurácia por Época')

    sns.lineplot(data=df_final, x='epochs', y='avg_distance', hue='sg', ax=axes[1])
    axes[1].set_title('Evolução da Distância Média por Época')

    plt.tight_layout()
    plt.show()

    # Boxplot (Distribuição das Distâncias)
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    sns.boxplot(data=df_final, x='sg', y='avg_distance', ax=axes[0])
    axes[0].set_title('Distribuição das Distâncias por SG')

    sns.boxplot(data=df_final, x='sg', y='accuracy', ax=axes[1])
    axes[1].set_title('Distribuição das Acurácias por SG')

    plt.tight_layout()
    plt.show()

    # Gráfico de Pareto (Impacto do Embedding)
    df_final['vector_size'] = df_final['vector_size'].astype(str)
    df_final['vector_size'] = df_final['vector_size'] + 'D'

    df_pareto = df_final.groupby('vector_size').agg({'accuracy': 'mean', 'avg_distance': 'mean'}).reset_index()

    df_pareto['accuracy'] = df_pareto['accuracy'] * 100
    df_pareto['avg_distance'] = df_pareto['avg_distance'] * 100

    df_pareto = df_pareto.sort_values(by='avg_distance', ascending=False)

    fig, ax = plt.subplots(figsize=(15, 6))

    sns.barplot(data=df_pareto, x='vector_size', y='avg_distance', color='red', ax=ax)
    ax.set_title('Impacto do Embedding na Distância Média')
    ax.set_ylabel('Distância Média (%)')
    ax.set_xlabel('Tamanho do Embedding')

    ax2 = ax.twinx()
    sns.lineplot(data=df_pareto, x='vector_size', y='accuracy', color='blue', ax=ax2)
    ax2.set_ylabel('Acurácia (%)')

    plt.tight_layout()
    plt.show()


def plot_loss(df_results):
    # Cria uma figura com 3 subplots (1 linha e 3 colunas)
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))  # 1 linha, 3 colunas

    # === 1. Histograma ===
    axes[0].hist(df_results['cosine_loss'], bins=20, color='skyblue', edgecolor='black')
    axes[0].set_xlabel('Cosine Loss')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Distribution of Loss for Analogies')

    # === 2. Gráfico de Linha ===
    axes[1].plot(df_results['cosine_loss'], color='red')
    axes[1].set_xlabel('Index of Analogy')
    axes[1].set_ylabel('Cosine Loss')
    axes[1].set_title('Loss for Each Analogy')

    # === 3. Boxplot ===
    sns.boxplot(x=df_results['cosine_loss'], ax=axes[2], color='lightgreen')
    axes[2].set_title('Boxplot of Loss for Analogies')

    # Ajusta o layout para que não haja sobreposição de textos
    plt.tight_layout()
    plt.show()

-------------------------------------------
## Inicialização
Aqui definimos lemos o arquivo do courpus, e definimos as pastas onde os modelos serão armazenados futuramente

In [ ]:
text8_path = './text8'
corpus = Text8Corpus(text8_path)

sentence = next(iter(corpus))
print(sentence[:15])

output_dir = './word2vec_models'
os.makedirs(output_dir, exist_ok=True)

output_dir1 = './word2vec_models_v1'
os.makedirs(output_dir1, exist_ok=True)

output_dir2 = './word2vec_models_v2'
os.makedirs(output_dir2, exist_ok=True)

-------------------------------------------
# Treinamento dos modelos Word2Vec

In [ ]:
# Hiperparâmetros para o GridSearch
param_grid = {
    'vector_size': [50, 100, 200],      # Tamanho do vetor de palavras
    'window': [3, 5, 7],                # Tamanho da janela de contexto
    'sg': [0, 1],                      # CBOW (0) ou Skip-gram (1)
    'epochs': [5, 10, 15],             # Número de iterações de treinamento
}

combinations = generate_hyperparameter_combinations(param_grid)

for i, params in enumerate(combinations):
    print(f"\nTreinando combinação {i+1}/{len(combinations)}: {params}")
    model = train_and_save_model(corpus, params, output_dir)

In [ ]:
# Hiperparâmetros para o GridSearch
param_grid1 = {
    'vector_size': [150, 250, 300],      # Tamanho do vetor de palavras
    'window': [3, 5, 7],                # Tamanho da janela de contexto
    'sg': [0, 1],                      # CBOW (0) ou Skip-gram (1)
    'epochs': [5, 10, 15],             # Número de iterações de treinamento
}

combinations1 = generate_hyperparameter_combinations(param_grid1)

for j, params in enumerate(combinations1):
    print(f"\nTreinando combinação {j+1}/{len(combinations1)}: {params}")
    model1 = train_and_save_model(corpus, params, output_dir1)

In [ ]:
# Hiperparâmetros para o GridSearch
param_grid2 = {
    'vector_size': [50, 100, 200],      # Tamanho do vetor de palavras
    'window': [3, 5, 7],                # Tamanho da janela de contexto
    'sg': [0, 1],                      # CBOW (0) ou Skip-gram (1)
    'epochs': [10, 20, 30],             # Número de iterações de treinamento
}

combinations2 = generate_hyperparameter_combinations(param_grid2)

for j, params in enumerate(combinations2):
    print(f"\nTreinando combinação {j+1}/{len(combinations2)}: {params}")
    model2 = train_and_save_model(corpus, params, output_dir2)

-------------------------------------------
## Avaliação dos modelos
### Modelo base

In [ ]:
questions_words_path = './questions-words.txt'

models = os.listdir(output_dir)
models = [model for model in models if model.endswith('.model')]

model_metrics = []
all_results = []

num_modelo = 0

for model_name in models:
    num_modelo += 1
    model = Word2Vec.load(os.path.join(output_dir, model_name))
    analogies = get_analogies(model, questions_words_path)
    accuracy, avg_distance, avg_loss, df_results = new_evaluate_models(model, analogies, model_name)
    
    print(f"Modelo: [{num_modelo}]", end='\r')
    
    model_metrics.append({
        'model_name': model_name,
        'accuracy': accuracy,
        'avg_distance': avg_distance,
        'vector_size': model.vector_size,
        'window': model.window,
        'sg': model.sg,
        'epochs': model.epochs
    })

    df_results['sg'] = 'CBOW' if model.sg == 0 else 'SkipGram'
    df_results['vector_size'] = model.vector_size
    df_results['window'] = model.window
    df_results['epochs'] = model.epochs

    #trocar todos os valores 0 de acuracia e avg_distance no df_results para esse modelo pelo seu valor real
    df_results['accuracy'] = accuracy
    df_results['avg_distance'] = avg_distance
    df_results['avg_loss'] = avg_loss
        
    # Armazenar o DataFrame na lista de todos os resultados
    all_results.append(df_results)

# Concatenar todos os DataFrames de resultados em um único DataFrame
final_df = pd.concat(all_results, ignore_index=True)

# Salvar o arquivo CSV de resultados na **raiz do projeto**
final_df.to_csv('./results/results.csv', index=False)

In [ ]:
df = pd.read_csv('./results/results.csv')
df.head()

### Modelo 1

In [ ]:
questions_words_path = './questions-words.txt'

models1 = os.listdir(output_dir1)
models1 = [model1 for model1 in models1 if model1.endswith('.model')]

model_metrics1 = []
all_results1 = []

num_modelo1 = 0

for model_name1 in models1:
    num_modelo1 +=1
    model1 = Word2Vec.load(os.path.join(output_dir1, model_name1))
    analogies1 = get_analogies(model1, questions_words_path)
    accuracy1, avg_distance1, avg_loss1, df_results1 = new_evaluate_models(model1, analogies1, model_name1)
    
    print(f"Modelo: [{num_modelo1}]", end='\r')
    
    model_metrics1.append({
        'model_name': model_name1,
        'accuracy': accuracy1,
        'avg_distance': avg_distance1,
        'vector_size': model1.vector_size,
        'window': model1.window,
        'sg': model1.sg,
        'epochs': model1.epochs
    })

    df_results1['sg'] = 'CBOW' if model1.sg == 0 else 'SkipGram'
    df_results1['vector_size'] = model1.vector_size
    df_results1['window'] = model1.window
    df_results1['epochs'] = model1.epochs

    #trocar todos os valores 0 de acuracia e avg_distance no df_results para esse modelo pelo seu valor real
    df_results1['accuracy'] = accuracy1
    df_results1['avg_distance'] = avg_distance1
    df_results1['avg_loss'] = avg_loss1
        
    # Armazenar o DataFrame na lista de todos os resultados
    all_results1.append(df_results1)

# Concatenar todos os DataFrames de resultados em um único DataFrame
final_df1 = pd.concat(all_results1, ignore_index=True)

# Salvar o arquivo CSV de resultados na **raiz do projeto**
final_df1.to_csv('./results/results1.csv', index=False)

In [ ]:
df1 = pd.read_csv('./results/results1.csv')
df1.head()

### Modelo 2

In [ ]:
questions_words_path = './questions-words.txt'

models2 = os.listdir(output_dir2)
models2 = [model2 for model2 in models2 if model2.endswith('.model')]

model_metrics2 = []
all_results2 = []

num_modelo2 = 0

for model_name2 in models2:
    num_modelo2 +=1
    model2 = Word2Vec.load(os.path.join(output_dir2, model_name2))
    analogies2 = get_analogies(model2, questions_words_path)
    accuracy2, avg_distance2, avg_loss2, df_results2 = new_evaluate_models(model2, analogies2, model_name2)
    
    print(f"Modelo: [{num_modelo2}]", end='\r')
    
    model_metrics2.append({
        'model_name': model_name2,
        'accuracy': accuracy2,
        'avg_distance': avg_distance2,
        'vector_size': model2.vector_size,
        'window': model2.window,
        'sg': model2.sg,
        'epochs': model2.epochs
    })

    df_results2['sg'] = 'CBOW' if model2.sg == 0 else 'SkipGram'
    df_results2['vector_size'] = model2.vector_size
    df_results2['window'] = model2.window
    df_results2['epochs'] = model2.epochs

    #trocar todos os valores 0 de acuracia e avg_distance no df_results para esse modelo pelo seu valor real
    df_results2['accuracy'] = accuracy2
    df_results2['avg_distance'] = avg_distance2
    df_results2['avg_loss'] = avg_loss2
        
    # Armazenar o DataFrame na lista de todos os resultados
    all_results2.append(df_results2)

# Concatenar todos os DataFrames de resultados em um único DataFrame
final_df2 = pd.concat(all_results2, ignore_index=True)

# Salvar o arquivo CSV de resultados na **raiz do projeto**
final_df2.to_csv('./results/results2.csv', index=False)

In [ ]:
df2 = pd.read_csv('./results/results2.csv')
df2.head()

In [153]:
#juntar os 3 df
df_final = pd.concat([df, df1, df2], ignore_index=True)
df_final.to_csv('./results/results_final.csv', index=False)

In [ ]:
#mostrar o top 10 dos melhores modelos de acordo com a distancia media
df_final.sort_values(by='avg_distance').head(5)


In [ ]:
#mostrar o top 10 dos melhores modelos de acordo com a acuracia
df_final.sort_values(by='accuracy', ascending=False).head(5)

-----------------------------
## Analise dos resultados
### Impacto da arquitetura: CBOW (Continuous Bag of Words) x Skip-gram

In [ ]:
plot_CBOWxSkipGram(df_final)

In [ ]:
plot_loss(df_results)

### Impacto do tamanho do embedding / vector_size - acurácia x distância de cosseno

In [ ]:
plot_embeddings_impact(df_final)

### Impacto da quantidade de épocas - 

# GPT


1. Histograma de Cosine Loss

    Como interpretar?
    O histograma mostra a distribuição de frequência dos valores de loss para todas as analogias testadas.

O que observar?

    Distribuição concentrada perto de zero:
        Se a maioria dos valores estiver próxima de zero, isso indica que as analogias estão sendo bem resolvidas.
        Um pico forte próximo de zero significa que muitas analogias têm vetores preditos próximos aos vetores esperados.

    Distribuição larga ou com muitos valores acima de 1:
        Isso sugere que o modelo tem dificuldade em resolver muitas analogias.
        Muitas perdas altas (>1) podem indicar que o modelo não aprendeu bem as relações semânticas entre as palavras.

    Outliers (valores fora do padrão):
        Outliers (valores muito acima de 1.5) indicam casos de analogias em que o vetor predito está muito distante do vetor correto.
        Pode ser interessante identificar quais analogias estão associadas a essas losses altas para entender onde o modelo falha.

    Exemplo de interpretação:
    Se o histograma mostrar uma distribuição que parece uma curva normal (em forma de sino) centrada em 0.2, isso significa que, em média, o erro entre os vetores esperados e preditos é pequeno. Se houver uma cauda longa (valores maiores que 1), isso indica algumas falhas significativas.

📈 2. Gráfico de Linha de Cosine Loss

    Como interpretar?
    O gráfico de linha mostra a evolução da loss para cada analogia, ou seja, cada ponto no eixo x representa uma analogia processada, e o eixo y mostra o valor da loss.

O que observar?

    Tendências ou picos:
        Se houver picos muito altos em algumas partes, o modelo pode estar falhando em um subconjunto específico de analogias.
        Se o gráfico estiver relativamente estável e baixo, isso é um bom sinal, pois mostra que o modelo está consistentemente resolvendo as analogias.

    Padrões Cíclicos:
        Se houver padrões repetidos (por exemplo, o erro aumenta e diminui periodicamente), isso pode indicar que o modelo está lidando de forma inconsistente com algumas classes de palavras.
        Isso pode acontecer se as analogias forem agrupadas por categorias (por exemplo, países e capitais ou palavras de gênero).

    Média geral:
        Se o gráfico está sempre abaixo de 0.5, significa que o erro médio de predição é pequeno, o que é positivo.
        Se a maior parte das losses estiver acima de 1, o modelo não está conseguindo capturar as relações esperadas entre as palavras.

    Exemplo de interpretação:
    Imagine que o gráfico mostre pequenos picos, mas que a maioria das losses está abaixo de 0.5. Isso indica que o modelo está consistentemente bem.
    Se o gráfico tiver várias spikes (picos) altos, pode ser útil verificar as palavras associadas a essas perdas e entender por que o modelo está falhando nessas analogias específicas.

📦 3. Boxplot de Cosine Loss

    Como interpretar?
    O boxplot mostra a distribuição e os outliers da loss de forma compacta. Ele exibe o mínimo, o primeiro quartil (Q1), a mediana, o terceiro quartil (Q3) e o máximo, além dos outliers (pontos fora do padrão).

O que observar?

    Mediana (linha no meio do boxplot):
        A posição da mediana indica a loss típica para o conjunto de analogias.
        Se a mediana for próxima de zero, é um bom sinal de que a maioria das analogias foi resolvida corretamente.

    Comprimento do "box" (Q1 a Q3):
        Se o box for estreito (entre Q1 e Q3), isso indica que a maioria das analogias tem perdas muito semelhantes.
        Se o box for largo, as losses estão muito espalhadas, o que pode indicar inconsistência do modelo.

    Outliers (pontos fora do boxplot):
        Outliers são analogias que tiveram losses muito acima do normal.
        Esses outliers podem ser causados por palavras fora do vocabulário ou relações semânticas que o modelo não aprendeu bem.
        Identificar quais analogias estão associadas a esses outliers pode ajudar a entender quais categorias de analogias o modelo tem dificuldade de resolver.